In [1]:
import pandas as pd
import numpy as np 
import datetime as dt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import scale
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In C:\Users\user\anaconda\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\user\anaconda\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\user\anaconda\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\user\anaconda\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\user\anaconda\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
Th

In [9]:
microclimate_data = pd.read_csv('precprocess_microclimate_data.csv')

In [20]:
microclimate_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6283 entries, 0 to 6282
Data columns (total 4 columns):
Modified_Date    6283 non-null object
Location         6283 non-null int64
New_Value        6283 non-null float64
Month            6283 non-null int64
dtypes: float64(1), int64(2), object(1)
memory usage: 196.5+ KB


In [14]:
microclimate_data.head()

,Modified_Date,Location,New_Value,Month
0,2020-02-13 05:00:00,7,7.2,2
1,2020-02-13 06:00:00,7,12.5,2
2,2020-02-13 07:00:00,7,5.9,2
3,2020-02-13 08:00:00,7,4.0,2
4,2020-02-13 09:00:00,7,5.3,2


In [12]:
microclimate_data.drop('Predictions',axis = 1,inplace = True)

In [15]:
mc_data_pred = microclimate_data[microclimate_data['Modified_Date'] <= '2020-09-15']

In [19]:
mc_data_pred['Modified_Date'] =  pd.to_datetime(mc_data_pred['Modified_Date'], infer_datetime_format=True)

C:\Users\user\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [23]:
mc_data_pred.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5155 entries, 0 to 5154
Data columns (total 8 columns):
Modified_Date    5155 non-null datetime64[ns]
Location         5155 non-null int64
New_Value        5155 non-null float64
Month            5155 non-null int64
year             5155 non-null int64
month            5155 non-null int64
day              5155 non-null int64
hour             5155 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(6)
memory usage: 362.5 KB


In [22]:
# extract the month and date in separate columns from the conversion columns
import datetime as dt
mc_data_pred['year'] = mc_data_pred['Modified_Date'].dt.year
mc_data_pred['month'] = mc_data_pred['Modified_Date'].dt.month
mc_data_pred['day'] = mc_data_pred['Modified_Date'].dt.day
mc_data_pred['hour'] = mc_data_pred['Modified_Date'].dt.hour

C:\Users\user\anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\user\anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\user\anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

In [25]:
mc_data_pred.drop(['Modified_Date','Month'],axis = 1,inplace = True)

C:\Users\user\anaconda\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [26]:
mc_data_pred.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5155 entries, 0 to 5154
Data columns (total 6 columns):
Location     5155 non-null int64
New_Value    5155 non-null float64
year         5155 non-null int64
month        5155 non-null int64
day          5155 non-null int64
hour         5155 non-null int64
dtypes: float64(1), int64(5)
memory usage: 281.9 KB


In [27]:
Y = mc_data_pred['New_Value'].values
X =mc_data_pred.drop('New_Value',axis=1).values

In [28]:
# Create training and test sets
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.2, random_state=42)

In [29]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
 
# Instantiate Linear Regression
linear_model = LinearRegression(normalize = True)
 
# Fit it to the  same split done in the previous part
linear_model.fit(X_train,y_train)
pred= linear_model.predict(X_test)

# Found the best parameters
print("Mean Absolute Error is:",mean_absolute_error(y_test,pred))

Mean Absolute Error is: 4.342683153898793


In [30]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
 
# Instantiate the ElasticNet regressor: elastic_net
elastic_net = ElasticNet(normalize = True)
 
# Create the hyperparameter grid
param_grid = {'alpha':[1e-5,1e-4,1e-3,1e-2,1e-1,1,2,3,4,5], 'l1_ratio':[1e-5,1e-4,1e-3,1e-2,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]}
 
# Setup the GridSearchCV object: using elastic net
gm_cv = GridSearchCV(elastic_net, param_grid, scoring='neg_mean_absolute_error' , cv=10)
 
# Fit it to the  same split done in the previous part
gm_cv.fit(X_train,y_train)
 
# Found the best parameters
print("Best Parameters: ",gm_cv.best_params_)
print("Mean Absolute Error is:",-gm_cv.best_score_)

Best Parameters:  {'alpha': 0.0001, 'l1_ratio': 0.4}
Mean Absolute Error is: 4.268025770809871


In [31]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
 
# Instantiate the ElasticNet regressor: elastic_net
Lasso_reg = Lasso(normalize = True)
 
# Create the hyperparameter grid
param_grid = {'alpha':[1e-5,1e-4,1e-3,1e-2,1e-1,1,2,3,4,5,10,15,20]}
 
# Setup the GridSearchCV object: using elastic net
gm_cv_lso = GridSearchCV(Lasso_reg, param_grid, scoring='neg_mean_absolute_error' , cv=10)
 
# Fit it to the  same split done in the previous part
gm_cv_lso.fit(X_train,y_train)
 
# Found the best parameters
print("Best Parameters: ",gm_cv_lso.best_params_)
print("Mean Absolute Error is:",-gm_cv_lso.best_score_)

Best Parameters:  {'alpha': 0.001}
Mean Absolute Error is: 4.271227498673565
